### WIP

In [ ]:
import os
import numpy as np
import pandas as pd 
import pickle
import pymysql
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

### Cargamos nuevo df procesado con nuevas noticias

In [3]:
current_dir = os.path.abspath(os.getcwd())
print(f"Directorio actual: {current_dir}")

# Subir hasta el directorio raíz común (en este caso 'Analisis-de-noticias')
root_dir = os.path.abspath(os.path.join(current_dir, "../../.."))
print(f"Directorio raíz común: {root_dir}")

# Construir la ruta al directorio donde están los archivos pkl
directory = os.path.join(root_dir, "src/00.data/preprocesado/")
print(f"Ruta de directorio ajustada: {directory}")

# Lista para almacenar los DataFrames
df_lista = []

# Buscar todos los archivos pkl en el directorio
archivos_pkl = [f for f in os.listdir(directory) if f.startswith("meneame_procesado_") and f.endswith(".pkl")]

# Leer cada archivo .pkl y agregarlo a la lista de DataFrames
for archivo in archivos_pkl:
    file_path = os.path.join(directory, archivo)
    with open(file_path, "rb") as f:
        df_chunk = pickle.load(f)
        df_lista.append(df_chunk)
        print(f"Cargado: {archivo} con {len(df_chunk)} filas")

# Concatenar todos los DataFrames en uno solo
df = pd.concat(df_lista, ignore_index=True)

# Verificar el tamaño del DataFrame final
print(f"DataFrame final con {df.shape[0]} filas y {df.shape[1]} columnas")

Directorio actual: C:\Users\Jordi\OneDrive - Finergal\Documentos\Bootcamp HAB Data Science\PFB\Analisis-de-noticias\src\00.data\SQL
Directorio raíz común: C:\Users\Jordi\OneDrive - Finergal\Documentos\Bootcamp HAB Data Science\PFB\Analisis-de-noticias
Ruta de directorio ajustada: C:\Users\Jordi\OneDrive - Finergal\Documentos\Bootcamp HAB Data Science\PFB\Analisis-de-noticias\src/00.data/preprocesado/
Cargado: meneame_procesado_1.pkl con 100000 filas
Cargado: meneame_procesado_2.pkl con 100000 filas
Cargado: meneame_procesado_3.pkl con 87588 filas
DataFrame final con 287588 filas y 19 columnas


In [4]:
df.head(3)

,news_id,title,content,full_story_link,meneos,clicks,karma,positive_votes,anonymous_votes,negative_votes,category,comments,published_date,user,source,source_link,scraped_date,provincia,comunidad
0,4032594,RTVE mantiene que Mazón llegó al Cecopi antes ...,Los servicios informativos de la radio y la te...,https://meneame.net/story/rtve-mantiene-mazon-...,115,651,410,57,58,1,Política y Sociedad,30,2025-02-27 12:50:02,CosasQueLees,eldiario.es,https://www.eldiario.es/comunitat-valenciana/r...,2025-02-27 13:30:08,NaN,NaN
1,4032568,Arde Mississippi - ¿De donde sale todo ese odio?,Gene Hackman en la película Arde Mississippi (...,https://meneame.net/story/arde-mississippi-don...,77,982,362,50,27,2,Entretenimiento y Cultura,24,2025-02-27 12:35:02,Grahml,youtube.com,https://www.youtube.com/watch?v=dnWDqcdbANQ,2025-02-27 13:30:08,NaN,NaN
2,4032569,La Guardia Civil desaloja todos los colegios d...,La Guardia Civil está desalojando preventivame...,https://meneame.net/story/guardia-civil-desalo...,97,761,369,50,47,0,Política y Sociedad,70,2025-02-27 12:10:05,Tunguska08Chelyabinsk13,levante-emv.com,https://www.levante-emv.com/sucesos/2025/02/27...,2025-02-27 13:30:08,NaN,NaN


### Cargamos df de Clustering para añadir info de Cluster

In [5]:
df = df.drop(columns=['full_story_link'], errors='ignore')

In [6]:
directorio_base = os.path.abspath(os.path.join(os.getcwd(), "../../.."))  # Subir dos directorios
directorio_pkl = os.path.join(directorio_base, "src", "00.data", "clustering")

# Obtener todos los archivos .pkl en el directorio de preprocesado
archivos_pkl = [f for f in os.listdir(directorio_pkl) if f.startswith("df_clustering_") and f.endswith(".pkl")]

# Si no se encuentran archivos .pkl, mostramos un mensaje
if not archivos_pkl:
    print("❌ No se encontraron archivos .pkl en el directorio.")
else:
    df_lista = []

    # Cargar todos los archivos .pkl
    for archivo in archivos_pkl:
        ruta_archivo = os.path.join(directorio_pkl, archivo)
        try:
            with open(ruta_archivo, "rb") as f:
                df_lista.append(pickle.load(f))
                print(f"✅ Archivo cargado correctamente: {archivo}")
        except FileNotFoundError:
            print(f"❌ No se encontró el archivo: {archivo}")
        except Exception as e:
            print(f"❌ Error al cargar {archivo}: {e}")

    # Concatenar todos los DataFrames en uno solo
    if df_lista:
        df_clustering = pd.concat(df_lista, ignore_index=True)
        print(f"✅ DataFrame final con {df_clustering.shape[0]} filas y {df_clustering.shape[1]} columnas")
    else:
        print("❌ No se cargaron DataFrames.")

✅ Archivo cargado correctamente: df_clustering_1.pkl
✅ Archivo cargado correctamente: df_clustering_2.pkl
✅ Archivo cargado correctamente: df_clustering_3.pkl
✅ DataFrame final con 287563 filas y 10 columnas


In [7]:
df_clustering.head(3)

,news_id,meneos,clicks,karma,positive_votes,negative_votes,anonymous_votes,comments,category,cluster_3
0,4032594,115,651,410,57,1,58,30,Política y Sociedad,1
1,4032568,77,982,362,50,2,27,24,Entretenimiento y Cultura,1
2,4032569,97,761,369,50,0,47,70,Política y Sociedad,1


In [8]:
# df_clustering["cluster_3"].value_counts()

### Conectamos con SQL

In [8]:
# Cargar las variables de entorno
load_dotenv()

user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("HOST", "localhost")
database = "meneame"

# Crear el engine de SQLAlchemy
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

### Cargamos todo en un df para comparar y quedarnos con las nuevas noticias en df_diff

In [9]:
# Extraer los datos de la tabla 'news_info_table' de la base de datos
query = "SELECT * FROM news_info_table"
df_db = pd.read_sql(query, engine)

In [10]:
df_db.shape

(287563, 17)

In [11]:
# Identificar las diferencias entre el DataFrame original y los datos en la base de datos
df_diff = df.merge(df_db[['news_id']], on="news_id", how="left", indicator=True)
df_diff = df_diff[df_diff['_merge'] == 'left_only'].drop(columns=['_merge'])
df_diff = df_diff.astype({'provincia': 'object', 'comunidad': 'object'})

In [12]:
# Reemplazar valores nulos por 'Desconocido'
df_diff['provincia'] = df_diff['provincia'].fillna('Desconocido')
df_diff['comunidad'] = df_diff['comunidad'].fillna('Desconocido')

In [17]:
def handle_new_data(df_diff):
    # Establecer conexión a la base de datos
    with engine.connect() as conn:
        for col in df_diff.columns:
            new_data = df_diff[col].unique()  # Obtener los nuevos valores de la columna
            
            for value in new_data:
                if col == 'user':
                    # Obtener el 'user_id' correspondiente del dataframe original
                    user_id = df[df['user'] == value]['user_id'].values[0]
                    # Insertar el nuevo usuario junto con su 'user_id'
                    conn.execute(text(f"INSERT INTO user_table (user_id, user) VALUES ({user_id}, '{value}')"))
                
                elif col == 'source':
                    conn.execute(text(f"INSERT INTO source_table (source) VALUES ('{value}')"))
                elif col == 'provincia':
                    conn.execute(text(f"INSERT INTO provincia_table (provincia) VALUES ('{value}')"))
                elif col == 'category':
                    # Asignar un 'category_id' con factorize
                    category_id = pd.factorize(df['category'])[0][df['category'] == value][0] + 1
                    # Insertar el nuevo valor de 'category' junto con su 'category_id'
                    conn.execute(text(f"INSERT INTO category_table (category_id, category) VALUES ({category_id}, '{value}')"))


In [18]:
# Ejecutar la función para procesar los nuevos valores y asegurarse de que los datos están actualizados
handle_new_data(df_diff)

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '2' for key 'category_table.PRIMARY'")
[SQL: INSERT INTO category_table (category_id, category) VALUES (2, 'Entretenimiento y Cultura')]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [ ]:
###############

### código para crear nueva tabla cluster e insertar la info de cluster en la tabla principal 

In [ ]:
# Crear la tabla 'cluster_table' si no existe
create_table_sql = """
CREATE TABLE IF NOT EXISTS cluster_table (
    cluster_id INT PRIMARY KEY,
    cluster_name VARCHAR(255)
);
"""

# Insertar los nombres de los clusters en la tabla 'cluster_table'
cluster_names = {
    0: "Noticias polémicas",
    1: "Noticias estándar",
    2: "Noticias populares"
}

# Ejecutar la creación de la tabla
with engine.connect() as connection:
    connection.execute(text(create_table_sql))  # Crear la tabla si no existe
    for cluster_id, cluster_name in cluster_names.items():
        # Insertar los datos en la tabla
        insert_sql = text(f"""
        INSERT INTO cluster_table (cluster_id, cluster_name)
        VALUES (:cluster_id, :cluster_name)
        ON DUPLICATE KEY UPDATE cluster_name = :cluster_name
        """)
        connection.execute(insert_sql, {"cluster_id": cluster_id, "cluster_name": cluster_name})

In [18]:
# Insertar la columna 'cluster_id' de df_clustering en 'news_info_table'
df_clustering['cluster_id'] = df_clustering['cluster_3'].map(cluster_names)

# Actualizar la tabla 'news_info_table' con el 'cluster_id' de df_clustering
for _, row in df_clustering.iterrows():
    update_query = f"""
    UPDATE news_info_table
    SET cluster_id = {row['cluster_id']}
    WHERE news_id = {row['news_id']};
    """
    with engine.connect() as connection:
        connection.execute(update_query)

print("Proceso completado con éxito.")

ObjectNotExecutableError: Not an executable object: '\n    UPDATE news_info_table\n    SET cluster_id = Noticias estándar\n    WHERE news_id = 4032594;\n    '

In [ ]:
######################

In [ ]:
# Cargar variables de entorno
load_dotenv()

# Obtener credenciales de la base de datos
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("HOST", "localhost")
database = "meneame"

# Conectar con SQLAlchemy para insertar DataFrames
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

# Conectar con pymysql para ejecutar sentencias SQL
try:
    connection = pymysql.connect(host=host, user=user, password=password, database=database)
    cursor = connection.cursor()

    # 1️⃣ Crear tabla `cluster_table` si no existe
    sql_create_cluster_table = """
    CREATE TABLE IF NOT EXISTS cluster_table (
        cluster_id INT PRIMARY KEY,
        cluster_name VARCHAR(50) NOT NULL
    );
    """
    cursor.execute(sql_create_cluster_table)

    # 2️⃣ Insertar los nombres de los clusters si no existen
    cluster_names = {
        0: "Noticias polémicas",
        1: "Noticias estándar",
        2: "Noticias populares"
    }
    
    for cluster_id, cluster_name in cluster_names.items():
        cursor.execute("""
            INSERT INTO cluster_table (cluster_id, cluster_name)
            VALUES (%s, %s) 
            ON DUPLICATE KEY UPDATE cluster_name = VALUES(cluster_name);
        """, (cluster_id, cluster_name))
    connection.commit()

    # 3️⃣ Extraer datos actuales de news_info_table
    sql_select_news = "SELECT * FROM news_info_table;"
    df_sql = pd.read_sql(sql_select_news, engine)
    
    # 4️⃣ Filtrar datos nuevos
    df_new = df[~df['news_id'].isin(df_sql['news_id'])]
    
    if not df_new.empty:
        df_new.to_sql("news_info_table", engine, if_exists="append", index=False)
        print("Nuevas noticias insertadas correctamente.")
    else:
        print("No hay nuevas noticias para insertar.")
    
    # 5️⃣ Asignar clusters a news_id
    df_clusters = df_clustering[['news_id', 'cluster_3']]
    
    for _, row in df_clusters.iterrows():
        cursor.execute("""
            UPDATE news_info_table 
            SET cluster_id = %s 
            WHERE news_id = %s;
        """, (row['cluster_3'], row['news_id']))
    
    connection.commit()
    print("Clusters asignados correctamente.")

except pymysql.MySQLError as e:
    print("Error en la conexión a MySQL:", e)

finally:
    cursor.close()
    connection.close()
